In [1]:
# TD9–TD10 : Mini-projet 
# Interface + filtres (type/auteur/année) + comparaison Reddit vs ArXiv
from TD4_5_corpus import Corpus
from TD7_moteur_recherche import SearchEngine
from td6_traitement_texte import corpus_stats, corpus_concorde  
import ipywidgets as widgets
from IPython.display import display
import re
#1) Charger le corpus complet
corpus = Corpus.load("data/corpus_complet.csv", nom="corpus_complet")
print("Corpus chargé :", corpus)

Corpus chargé : Corpus(corpus_complet, ndoc=10, naut=10)


In [ ]:
# 2) Créer le moteur (TD7)
moteur = SearchEngine(corpus)
# 3) type/auteur/année
def get_doc_type(doc):
    # doc.get_type() existe(reddit/arxiv/document)
    if hasattr(doc, "get_type"):
        return doc.get_type()
    # fallback si jamais
    return "document"
def get_doc_year(doc):
    d = str(getattr(doc, "date", "")).strip()
    m = re.search(r"(\d{4})", d)
    return int(m.group(1)) if m else None
# préparer listes pour les filtres
types = sorted(list({get_doc_type(d) for d in corpus.id2doc.values()}))
auteurs = sorted(list({str(d.auteur) for d in corpus.id2doc.values()}))
years = [get_doc_year(d) for d in corpus.id2doc.values()]
years = [y for y in years if y is not None]
ymin, ymax = (min(years), max(years)) if years else (2000, 2025)
# petit "corpus" pour stats/concorde sur une sélection
class MiniCorpus:
    def __init__(self, id2doc):
        self.id2doc = id2doc
def ids_filtrés(type_sel="Tous", auteur_sel="Tous", year_min=None, year_max=None):
    ids = []
    for doc_id, doc in corpus.id2doc.items():
        t = get_doc_type(doc)
        a = str(doc.auteur)
        y = get_doc_year(doc)
        if type_sel != "Tous" and t != type_sel:
            continue
        if auteur_sel != "Tous" and a != auteur_sel:
            continue
        if year_min is not None and y is not None and y < year_min:
            continue
        if year_max is not None and y is not None and y > year_max:
            continue
        ids.append(doc_id)
    return ids


100%|██████████| 10/10 [00:00<00:00, 3973.76it/s]


In [3]:
# 4)Widgets interface
titre = widgets.HTML("<h3 style='margin:0'>Mini-projet : Exploration du corpus </h3>")
# filtres communs
filtre_type = widgets.Dropdown(options=["Tous"] + types, value="Tous", description="Type :")
filtre_auteur = widgets.Dropdown(options=["Tous"] + auteurs, value="Tous", description="Auteur :")
filtre_annee = widgets.IntRangeSlider(
    value=[ymin, ymax], min=ymin, max=ymax, step=1,
    description="Années :",
    continuous_update=False,
    layout=widgets.Layout(width="520px")
)
# sorties
out_search = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="10px", height="260px", overflow_y="auto"))
out_conc   = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="10px", height="260px", overflow_y="auto"))
out_stats  = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="10px", height="260px", overflow_y="auto"))
out_comp   = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="10px", height="260px", overflow_y="auto"))


In [ ]:
# Onglet 1 : Recherche TD7 (avec filtrage)
q = widgets.Text(description="Requête :", placeholder="ex : photo camera", layout=widgets.Layout(width="520px"))
k = widgets.IntSlider(value=5, min=1, max=30, description="Top K :", continuous_update=False)
btn_search = widgets.Button(description="Rechercher", button_style="primary", icon="search")
def do_search(b):
    out_search.clear_output()
    with out_search:
        requete = q.value.strip()
        if requete == "":
            print("Veuillez entrer une requête.")
            return
        # 1) on récupère plus de résultats que nécessaire (pour filtrer ensuite)
        res = moteur.search(requete, max(50, k.value))
        # 2) appliquer filtres type/auteur/année sur les doc_id du résultat
        tsel = filtre_type.value
        asel = filtre_auteur.value
        y0, y1 = filtre_annee.value
        def ok(doc_id):
            doc = corpus.id2doc[int(doc_id)]
            if tsel != "Tous" and get_doc_type(doc) != tsel:
                return False
            if asel != "Tous" and str(doc.auteur) != asel:
                return False
            y = get_doc_year(doc)
            if y is not None and (y < y0 or y > y1):
                return False
            return True
        res = res[res["doc_id"].apply(ok)]
        # 3) enlever scores 0 
        if "score" in res.columns:
            res = res[res["score"] > 0]
        # 4) garder top K après filtre
        res = res.head(k.value)
        if len(res) == 0:
            print("Aucun résultat après filtrage.")
        else:
            display(res)
btn_search.on_click(do_search)
tab_search = widgets.VBox([
    widgets.HTML("<b>TD7 : Moteur de recherche (avec filtres)</b>"),
    q,
    k,
    btn_search,
    out_search
])

In [5]:
# Onglet 2 : Concorde TD6 
motif = widgets.Text(description="Motif :", placeholder="ex : photo", layout=widgets.Layout(width="520px"))
ctx = widgets.IntSlider(value=20, min=5, max=80, description="Contexte :", continuous_update=False)
btn_conc = widgets.Button(description="Concorde", button_style="info", icon="list")
def do_conc(b):
    out_conc.clear_output()
    with out_conc:
        m = motif.value.strip()
        if m == "":
            print("Veuillez entrer un motif.")
            return
        tsel = filtre_type.value
        asel = filtre_auteur.value
        y0, y1 = filtre_annee.value
        ids = ids_filtrés(tsel, asel, y0, y1)
        sub = MiniCorpus({i: corpus.id2doc[i] for i in ids})
        dfc = corpus_concorde(sub, m, contexte=ctx.value)
        if len(dfc) == 0:
            print("Aucune occurrence trouvée.")
        else:
            display(dfc.head(30))
btn_conc.on_click(do_conc)
tab_conc = widgets.VBox([
    widgets.HTML("<b>TD6 : Concordancier</b>"),
    motif,
    ctx,
    btn_conc,
    out_conc
])


In [6]:
# Onglet 3 : Stats TD6 
topn = widgets.IntSlider(value=10, min=5, max=50, description="Top N :", continuous_update=False)
btn_stats = widgets.Button(description="Stats", button_style="warning", icon="bar-chart")
def do_stats(b):
    out_stats.clear_output()
    with out_stats:
        tsel = filtre_type.value
        asel = filtre_auteur.value
        y0, y1 = filtre_annee.value
        ids = ids_filtrés(tsel, asel, y0, y1)
        sub = MiniCorpus({i: corpus.id2doc[i] for i in ids})
        freq = corpus_stats(sub, n=topn.value, avec_doc_freq=True)
        display(freq.head(topn.value))
btn_stats.on_click(do_stats)
tab_stats = widgets.VBox([
    widgets.HTML("<b>TD6 : Statistiques</b>"),
    topn,
    btn_stats,
    out_stats
])


In [ ]:
# Onglet : Comparaison Reddit vs ArXiv
btn_comp = widgets.Button(description="Comparer reddit vs arxiv", button_style="success", icon="exchange")
def do_comp(b):
    out_comp.clear_output()
    with out_comp:
        y0, y1 = filtre_annee.value
        # comparer sur la même période, sans filtre auteur 
        ids_reddit = ids_filtrés("reddit", "Tous", y0, y1)
        ids_arxiv  = ids_filtrés("arxiv", "Tous", y0, y1)
        sub_r = MiniCorpus({i: corpus.id2doc[i] for i in ids_reddit})
        sub_a = MiniCorpus({i: corpus.id2doc[i] for i in ids_arxiv})
        print("Docs reddit :", len(ids_reddit), " VS Docs arxiv :", len(ids_arxiv))
        print("\nTop mots reddit :")
        fr = corpus_stats(sub_r, n=10, avec_doc_freq=True)
        display(fr.head(10))
        print("\nTop mots arxiv :")
        fa = corpus_stats(sub_a, n=10, avec_doc_freq=True)
        display(fa.head(10))
btn_comp.on_click(do_comp)
tab_comp = widgets.VBox([
    widgets.HTML("<b>TD9 :  Comparaison </b>"),
    widgets.HTML("Cette partie compare automatiquement reddit et arxiv (même période)."),
    btn_comp,
    out_comp
])


In [ ]:
# 5) Affichage: abs + layout
tabs = widgets.Tab(children=[tab_search, tab_conc, tab_stats, tab_comp])
tabs.set_title(0, "Recherche")
tabs.set_title(1, "Concorde")
tabs.set_title(2, "Stats")
tabs.set_title(3, "Comparaison")
filtres_box = widgets.VBox([
    widgets.HTML("<b>Filtres</b>"),
    widgets.HBox([filtre_type, filtre_auteur]),
    filtre_annee
])
display(widgets.VBox([titre, filtres_box, tabs]))
